In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import random
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from datetime import datetime

In [8]:
#chrome = webdriver.Chrome(ChromeDriverManager().install())
chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
chrome.get("https://web.whatsapp.com")

In [18]:
#Recherche le chemin passé en arument et le renvoie
#Si rien n'est trouvé avant le timeout, une exception est levée.
def chercher_element(chemin: str, timeout: int):
    trouve = False
    temps = 0
    while(not trouve):
        try: 
            element = chrome.find_element("xpath", chemin)
            trouve = True
        except: 
            time.sleep(0.5)
            temps += 0.5
        if(temps > timeout):
            raise Exception("Élément non trouvé pour le chemin : " + chemin) 
    return element

#Recherche tous les chemins passés en arument et renvoie le premier élément trouvé, ainsi que l'index du chemin auquel il correspond.
#Si rien n'est trouvé avant le timeout, une exception est levée.
#On n'utilise pas la fonction chercher_element afin de faire toutes les recherches en parallèle pour ne pas attendre les timeout à chaque fois.
def chercher_elements(chemins, timeout: int):
    trouve = False
    temps = 0
    element = None
    index = -1
    while(temps < timeout):
        for i in range(len(chemins)):
            try: 
                index = i
                element = chrome.find_element("xpath", chemins[i])
                trouve = True
                return (element, index)
            except:
                time.sleep(0.3)
                temps += 0.3
    raise Exception("Élément non trouvé pour les chemins : " + str(chemins))
    

def taper_message(tel: str, nom:str, prenom: str, message: str, num_contact: str, mots_cle_interdits = []):
    
    #taper le message dans le champ message
    #Le caractère '\n' est interprété comme un appui sur la touche "entrer". Ce qui envoie le message. On remplace tous les \n par une combinaison Shift + Enter
    search_box = chercher_element('//footer//div[@contenteditable="true"][@dir="ltr"][@spellcheck="true"][@role="textbox"][@title="Taper un message"][@data-testid="conversation-compose-box-input"]', 3)
    
    
    for ligne in message.split('\n'):
        search_box.send_keys(ligne)
        ActionChains(chrome).key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
    return True
    

#Envoie un message au numéro de téléphone passé en argument, si celui ci est trouvé.
#La chaine de la PJ doit être un chemin absolu vers un fichier présent en local sur l'ordinateur.
#Pour ne pas mettre de PJ, simplement passer une chaine vide, ou bien ne pas préciser cet argument.
#mots_cle_interdits est un tableau de chaines de caractère. S'il n'est pas vide, le message ne sera envoyé que si la conversation sélectionnée ne contient aucun des mots clé interdits. (permet d'éviter les messages en doublon, mais prend du temps)
def envoyer_message(tel: str, nom: str, prenom: str, message: str, image='', mots_cles_interdits = []):
    global statut_envoi
    
    #On commence par chercher si un des mots-clé interdits existe dans la conversation.
    time.sleep(1) #On laisse le temps à la conversation de se charger. Sinon, la recherche n'a pas accès aux anciens messages.
    try:
        doit_envoyer = True
        for mot_cle in mots_cles_interdits :
            if(chercher_contenu_dans_discussion(mot_cle)):
                doit_envoyer = False
                statut_envoi += " | le mot-clé interdit '" + mot_cle + "' a été trouvé -> pas d'envoi"
                return False
        if(doit_envoyer):
            statut_envoi += " | aucun des mots-clés interdits n'a été trouvé -> envoi"
            print(statut_envoi)
    except Exception as e:
        print(str(e))
        statut_envoi += " | exception lors de la recherche des mots-clés"
        print(statut_envoi)
        raise Exception("exception lors de la recherche des mots-clés")
    
    #Aucun des mots-clé interdits n'a été trouvé, on continue
    if(not taper_message(tel, nom, prenom, message, num_contact, mots_cles_interdits)):
        return False
    time.sleep(0.5)
    
    if(image == ""):
        bouton_envoyer = chercher_element("//button[span/@data-testid='send']", 1)
        bouton_envoyer.click()
    else:
        #cliquer sur le bouton joindre
        boutonJoindre = chercher_element("//div[@role='button'][@title='Joindre']", 1)
        boutonJoindre.click()

        #sélection du fichier à envoyer
        input_box = chrome.find_element(By.tagName('input'))
        input_box.send_keys(image)

        #Attendre que le fichier se charge
        bouton_envoyer = chercher_element("//div[@role='button'][span/@data-testid='send']", 35)
        time.sleep(1)
        bouton_envoyer.click()
    
    statut_envoi += "| envoyé"
    return True


#Recherche la chaine passée en argument dans l'historique des message du contact actuellement sélectionné.
#Renoie vrai si au moins un message contient la chaine, faux sinon.
def chercher_contenu_dans_discussion(contenu: str):
    global statut_envoi
    try:
        bouton_chercher = chercher_element('//div[@role="button"][@title="Recherche..."]', 2)
        bouton_chercher.click()
    except Exception as e:
        print(e)
    try:
        champ_saisie = chercher_element('//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::*//div[@contenteditable="true"][@class="_13NKt copyable-text selectable-text"]', 3)
        champ_saisie.clear()
        time.sleep(0.25)
        champ_saisie.send_keys(contenu)
        time.sleep(0.5)
    except Exception as e:
        statut_envoi += " | echec de la sélection de la barre de recherche"
        print(statut_envoi)
        raise Exception("echec de la sélection de la barre de recherche : " + str(e)) 
    
    chemin_succes = '/html/body/div[1]/div/div/div[2]/div[3]/span/div/div/div[2]/div[1]/div/div/div/div/div/div[2]/div[2]/div[1]/span/span/strong[@class="i0jNr"][text()="' + contenu + '"]'
    chemin_succes2 ='//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::div//div[@class="_3uIPm WYyr1"]/div[@class="_3m_Xw"]/div/div/div/div/div/span[@class="Hy9nV"][contains(@title, "' + contenu + '")]'
    chemin_echec  = '//header[//*[contains(text(),"Rechercher des messages")]]/following-sibling::div[@id="pane-side"]/div/div/span[@dir="auto"][@class="i0jNr"][text()="Aucun message trouvé"]'

    index = -1
    (resultat, index) = chercher_elements([chemin_succes, chemin_succes2, chemin_echec], 10)
    if(index == 0 or index == 1):
        statut_envoi += " | message vraiment trouvé"
        return True
    elif((index == 2)):
        statut_envoi += " | message vraiment non trouvé"
        return False
    else:
        statut_envoi += " | valeur index = " + str(index) + "non acceptée dans la recherche de message"
        return True
    champ_saisie.clear()

    #Si rien n'a été renvoyé à ce moment, c'est qu'il y a eu un problème -> Levée d'exception.
    statut_envoi += " | échec de la recherche"
    raise Exception("Échec de la recherche pour le contenu " + contenu) 


    
#Recherche, puis sélectionne le contact s'il est trouvé. S'il n'est pas trouvé, renvoie Faux
#Si le contact est trouvé, vérifie qu'aucun des messages passés ne contient un des mot_clés interdits (pour éviter les envois en doublon)
#Au final, ne renvoie vrai que si le contact a été correctement sélectionné et qu'il faut envoyer le message.
def selectionner_contact(tel: str, nom: str, prenom: str, num_contact: str, mots_cles_interdits = []):
    global statut_envoi
    #On commence par cliquer sur le bouton pour annuler la recherche précédente
    try:
        bouton_annuler_recherche = chrome.find_element("xpath", '//button[@aria-label="Annuler la recherche"]')
        bouton_annuler_recherche.click()
        #print("recherche annulée avec succès !")
        time.sleep(0.5)
    except Exception as e:
        print(str(e))
    #On recherche le champ de saisie de la recherche de contact, pour y taper le numéro de téléphone.
    try:
        search_box = chrome.find_element("xpath", '//div/div/div[@role="textbox"][@contenteditable="true"][@data-testid="chat-list-search"][not(@spellcheck)][@title="Champ de recherche"]')
        search_box.clear()
        search_box.send_keys(tel)
    except Exception as e:
        print(str(e))
    time.sleep(0.5)
    #Xpath1 ne comporte pas nativement de méthode pour passer en casse majuscule. On fait ça à la main.
    #Maintenant qu'on a tapé le numéro de téléphone dans le champ de recherche, on cherche si un titre de conversation apparaît
    #Si le message "Aucun contact, discussion ou message trouvé" apparaît, cela signifie que cette personne n'a pas whatsapp.
    try:
        titre_UpCass = 'translate(@title,"abcdefghijklmnopqrstuvwxyzèéêëïîäâàüûùöô","ABCDEFGHIJKLMNOPQRSTUVWXYZÈÉÊËÏÎÄÂÀÜÛÙÖÔ")'
        #chaine_chemin_contact = '//div[@aria-label="Résultats de la recherche."]/div[//span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")][not(contains(@title, "fait également partie"))]]'
        chaine_chemin_contact = '//div[@aria-label="Résultats de la recherche."]/div[//div[@data-testid="cell-frame-title"]/span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")][not(contains(@title, "fait également"))]]'
        chaine_chemin_contact2 = '//div[@aria-label="Résultats de la recherche."]/div/div[//div[@data-testid="cell-frame-title"]/span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")][not(contains(@title, "fait également"))]]'
        chaine_chemin_contact3 = '//div[@aria-label="Résultats de la recherche."]/div/div/div[//div[@data-testid="cell-frame-title"]/span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")][not(contains(@title, "fait également"))]]'
        chaine_chemin_contact4 = '//div[@aria-label="Résultats de la recherche."]/div/div/div/div[//div[@data-testid="cell-frame-title"]/span[contains(' + titre_UpCass + ',"{0}") or contains(' + titre_UpCass + ',"{1}")][not(contains(@title, "fait également"))]]'
        
        xpath_contact_trouve = chaine_chemin_contact.format(nom.upper(), prenom.upper())
        xpath_contact_trouve2 = chaine_chemin_contact2.format(nom.upper(), prenom.upper())
        xpath_contact_trouve3 = chaine_chemin_contact3.format(nom.upper(), prenom.upper())
        xpath_contact_trouve4 = chaine_chemin_contact4.format(nom.upper(), prenom.upper())
        xpath_contact_non_trouve = '//span[@dir="auto"][@class="i0jNr"][contains(text(),"Aucun contact, discussion ou message trouvé")]'
    except Exception as e:
        print(str(e))
    print(0)
    try:
        index = -1
        (bouton_contact, index) = chercher_elements([xpath_contact_trouve, xpath_contact_non_trouve], 3)
        print(1)
        if(index == 1):
            statut_envoi += " | Contact officiellement non trouvé"
            print("contact officiellement non trouvé")
            raise Exception("contact officiellement non trouvé")
        elif((index == -1)):
            statut_envoi += "échec de la recherche du contact"
            print("échec de la recherche du contact")
            raise Exception("échec de la recherche du contact")
        print(2)
        try:
            xpath_contact_trouves = [chaine_chemin_contact.format(nom.upper(), prenom.upper()),
                                     chaine_chemin_contact2.format(nom.upper(), prenom.upper()),
                                     chaine_chemin_contact3.format(nom.upper(), prenom.upper()),
                                     chaine_chemin_contact4.format(nom.upper(), prenom.upper())]
            bouton_contacts = [chercher_element(xpath_contact_trouve, 1),
                              chercher_element(xpath_contact_trouve2, 1),
                              chercher_element(xpath_contact_trouve3, 1),
                              chercher_element(xpath_contact_trouve4, 1)]

        except Exception as e:
            print("erreur lors de la recherche des boutons alternatifs : " + str(e))
        print(3)
        #Il semble que l'élément cliquable ne soit pas le même d'une conversation à l'autre. Je clique donc sur tout ce que je peux en espérant trouver le bon.
        temps_max = 3
        temps = 0       
        trouve = False
        
        prenom = prenom.upper().replace(" ", "").replace("-", "").replace("'", "")
        nom = nom.upper().replace(" ", "").replace("-", "").replace("'", "")
        print(4)
        while(not(trouve) and (temps < temps_max)):
            print("dans le while recherche de la conversation")
            for num_bouton in range(len(bouton_contacts)):
                try:
                    print("click sur le bouton " + str(num_bouton) + "   temps = " + str(temps))
                    bouton_contacts[num_bouton].click()
                    time.sleep(0.5)
                    Titre_conversation = chercher_element("//header/div/div/div/span[@dir='auto']", 4).text
                    Titre_conversation = Titre_conversation.upper().replace(" ", "").replace("-", "").replace("'", "")
                    trouve = (prenom in Titre_conversation) or (nom in Titre_conversation) or (str(tel) in Titre_conversation) or (("N" + num_contact) in Titre_conversation)
                    print("Titre_conversation" + Titre_conversation)
                    print("prenom : " + prenom)
                    print("prenom in Titre_conversation : " + str(prenom in Titre_conversation))
                    print("nom : " + nom)
                    print("nom in Titre_conversation : " + str(nom in Titre_conversation))
                    print("tel : " + str(tel))
                    print("str(tel) in Titre_conversation)" + str(str(tel) in Titre_conversation))
                    print("('N' + num_contact) in Titre_conversation : " + str((("N" + num_contact) in Titre_conversation)))
                    if(trouve):
                        print("Le titre correspond !")
                        break #On sort de la boucle for et le while va sortir en même temps.
                except Exception as e:
                    print(str(e))
            temps = temps + 1
        #print("après le while trouvé = " + str(trouve))
        print(5)
        search_box.clear()
        statut_envoi += " | contact trouvé 0"
        
        Titre_conversation = chercher_element("//header/div/div/div/span[@dir='auto']", 4).text

        print("Titre conversation : " + Titre_conversation)
        statut_envoi += " | 2"
        Titre_conversation = Titre_conversation.upper().replace(" ", "").replace("-", "").replace("'", "")
        statut_envoi += " | 2.1"
        if((prenom in Titre_conversation) or (nom in Titre_conversation) or (tel in Titre_conversation) or (("N" + num_contact) in Titre_conversation)):
            statut_envoi += " | contact sélectionné"
        else:
            print("Mauvais titre")
            statut_envoi += " | mais mauvais titre : '" + nom + " " + prenom + "' titre conversation : '" + Titre_conversation + "'"
            print(statut_envoi)
            raise Exception("mauvais titre : '" + nom + " " + prenom + "' titre conversation : '" + Titre_conversation + "'")

    except Exception as e:
        print(str(e))
        statut_envoi += " | Contact non trouvé"
        print(statut_envoi)
        raise Exception("Contact non trouvé")

    
    
def generer_message(prenom, nom, lycee, ville, annee):
    if(lycee != ""):
        lycee = "au lycée " + lycee
    else : 
        lycee = "dans ton lycée"
        
    #message = "Bonjour {0} {1},\n\n" + \
    #          "J'espère que tout s'e"
    #message = message.format(prenom, nom, lycee, ville)
        
    if(annee == "2020"):
        message = "Bonjour {0} {1},\n\n" + \
                  "Le premier concours d'entrée à l'IFNTI aura lieu *ce mercredi 29 juin*. Il ne reste que quelques jours pour s'inscrire.\n\n" + \
                  "Il y aura une autre session après les résultats du BAC. Mais ce sera plus difficile car il restera moins de places disponibles...\n\n" + \
                  "Cliquer ici pour s'inscrire : *https://forms.gle/9Cvqz7qpobzFgQuN7*"
        
        message = message.format(prenom, nom)
        
    elif(annee == "2021"):
        message = "Bonjour {0} {1},\n\n" + \
                  "Le premier concours d'entrée à l'IFNTI aura lieu *ce mercredi 29 juin*. Il ne reste que quelques jours pour s'inscrire.\n\n" + \
                  "Il y aura une autre session après les résultats du BAC. Mais ce sera plus difficile car il restera moins de places disponibles...\n\n" + \
                  "Cliquer ici pour s'inscrire : *https://forms.gle/9Cvqz7qpobzFgQuN7*"
        message = message.format(prenom, nom, lycee, ville)
        
    elif(annee == "2022"):
        message = "Bonjour {0} {1},\n\n" + \
                  "J'espère que tout s'est bien passé pour les épreuves du BAC2.\n\n" + \
                  "Le premier concours d'entrée à l'IFNTI aura lieu *ce mercredi 29 juin*. Il ne reste que quelques jours pour s'inscrire.\n\n" + \
                  "Il y aura une autre session après les résultats du BAC. Mais ce sera plus difficile car il restera moins de places disponibles...\n\n" + \
                  "Cliquer ici pour s'inscrire : *https://forms.gle/9Cvqz7qpobzFgQuN7*"
        message = message.format(prenom, nom, lycee, ville)
    else:
        print("erreur : étudiant avec année différent de 2020, 2021 ou 2022 : '" + str(annee) +"'")
        message = ""
    return message


statut_envoi = ""
fichier_etudiants = 'liste_etudiants_interessesSabManaarMurielHegra.csv'
fichier_log = 'log_envoi.csv'
with open(fichier_etudiants, "r") as filin:
    with open(fichier_log, 'a') as filout:
        num_contact_envoye = 0
        for ligne in filin:
            statut_envoi = ""
            ligne = ligne.split(",")
            num_contact = ligne[0].replace("\n", "")
            ville = ligne[1].replace("\n", "")
            lycee = ligne[2].replace("\n", "")
            nom = ligne[3].replace("\n", "")
            prenom = ligne[4].replace("\n", "")
            prefixe_tel = ligne[5].replace("\n", "")
            tel = ligne[6].replace("-", "").replace("\n", "")
            tel2 = ligne[7].replace("-", "").replace("\n", "")
            annee = ligne[9].replace("\n", "")

            
            
            piece_jointe = ""
            #piece_jointe = "/home/jean-christophe/git_IFNTI/Administration/Outils_de_com/Videos/Presentation_IFNTI_juin_2021/Presentation_IFNTI_juin_2021_VLQ.mp4"
            #piece_jointe = "/home/jean-christophe/Images/vidage_telephone/2022/05-mai/20220521_130250.jpg"
            #piece_jointe = ""
            now = datetime.now().strftime("%d/%m/%Y,%H:%M:%S")
            
            #Pour ne pas mettre de pièce jointe, simplement mettre une chaine vide
            for num_tel in {tel, tel2}:
                if(num_tel != ""):
                    print(num_tel)
                    statut_envoi = ""
                    try:
                        selectionner_contact(num_tel, nom, prenom, num_contact)
                        message0 = generer_message(prenom, nom, lycee, ville, annee)
                        #envoyer_message(num_tel, nom, prenom, message0, piece_jointe, ["Le premier concours d'entrée"])
                        envoyer_message(num_tel, nom, prenom, message0, piece_jointe, [])
                        message1 = "Vous cherchez une formation supérieure en informatique ? Découvrez l'IFNTI à Sokodé !\n\n" + \
                                  "*Présentation générale*\n" + \
                                  "L'objectif de l'Institut de Formation aux Normes et Technologies de l'Informatique est de faire en sorte que les étudiants togolais et plus particulièrement de la région centrale puissent bénéficier à moindre coût d'une formation supérieure en informatique, au standard international. Installé à Sokodé au cœur du Togo, l'IFNTI recrute les bacheliers scientifiques pour une licence professionnelle LMD en 3 ans.\n" + \
                                  "L'IFNTI est situé entre le lycée Technique et le lycée Moderne, dans le quartier administratif.\n\n" + \
                                  "*Conditions d'enseignement*\n" + \
                                  "La qualité de la formation est liée au taux d‘encadrement, avec de petits effectifs (promotions de 24 étudiants), et aux enseignants permanents de niveau ingénieur. Ceux-ci sont toujours disponibles pour soutenir les apprenants. Les modalités didactiques employées sont très proches de l’activité professionnelle. Depuis l'ouverture de l'IFNTI en 2014, tous les diplômés travaillent.\n\n" + \
                                  "*Candidature à l'IFNTI*\n" + \
                                  "Les promotions étant limitées à 24 étudiants, l'entrée à l'IFNTI se fait sur concours." + \
                                  "L'inscription se fait en ligne, en remplissant ce formulaire : https://forms.gle/9Cvqz7qpobzFgQuN7\n" + \
                                  "Il se déroule en deux étapes :\n" + \
                                  " - des épreuves écrites pendant une journée.\n" + \
                                  " - un entretien oral, conditionné par la réussite des épreuves écrites.\n" + \
                                  "Les deux étapes se dérouleront dans nos locaux à Sokodé.\n" + \
                                  "Vous serez évalués sur trois matières :\n" + \
                                  "- Français\n" + \
                                  "- Anglais\n" + \
                                  "- Mathématiques\n" + \
                                  "Frais d'inscription au concours : 10.000 FCFA.\n" + \
                                  "Date du premier *concours : 29 juin*. Une autre session du concours aura lieu après les résultats du bac2.\n\n" + \
                                  "*Inscription académique*\n" + \
                                  "Si vous êtes retenus après les épreuves orales, vous pourrez commencer la procédure d'inscription académique.\n" + \
                                  "Frais de scolarité : 590.000 FCFA / an.\n" + \
                                  "Frais d'inscription : 30.000 FCFA / an.\n" + \
                                  "Ces frais restent les mêmes pendant toute la formation. Les frais d'inscription ne sont pas remboursables.\n\n" + \
                                  "*Hébergement*\n" + \
                                  "L'IFNTI ne dispose pas de logements propres. Toutefois il est très facile de trouver un logement à Sokodé, et à un faible coût.\n\n" + \
                                  "Site internet\n" + \
                                  "https://www.ifnti.com\n\n" + \
                                  "Whatsapp\n" + \
                                  "+228 90 91 81 41\n\n" + \
                                  "Email\n" + \
                                  "contact@ifnti.com\n\n" + \
                                  "Vidéo de présentation\n" + \
                                  "https://www.youtube.com/watch?v=oDosebpBEf8"
                        #envoyer_message(num_tel, nom, prenom, message1, piece_jointe, ["Vous cherchez une formation"])
                        #message2 = "Voici la vidéo de présentation de l'IFNTI."
                        #piece_jointe = "/home/jean-christophe/git_IFNTI/Administration/Outils_de_com/Videos/Presentation_IFNTI_juin_2021/Presentation_IFNTI_juin_2021_VLQ.mp4"
                        #envoyer_message(num_tel, nom, prenom, message2, piece_jointe, ["Voici la vidéo de"])
                        print("\n\n\n")
                    except Exception as e:
                        print(str(e))
                    filout.write(now + "," + num_contact + "," + ville + "," + lycee + "," + nom + "," + prenom + "," + prefixe_tel + "," + num_tel + ",,," + annee + "," + statut_envoi + ",\n")
                    #Le fichier de Log a été organisé en suivant la structure du fichier liste_etudiants_intéresses.csv, de façon à pouvoir relancer le script sur tous ceux pour qui l'envoi a échoué afin de tenter une deuième fois.
                        
filout.close() 
filin.close()
print("Terminé.")

99951616
0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversationIFNTIFINDÉTUDES
prenom : MURIEL
prenom in Titre_conversation : False
nom : MILHAU
nom in Titre_conversation : False
tel : 99951616
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 1   temps = 0
Titre_conversationIFNTIFINDÉTUDES
prenom : MURIEL
prenom in Titre_conversation : False
nom : MILHAU
nom in Titre_conversation : False
tel : 99951616
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 2   temps = 0
Titre_conversationIFNTIFINDÉTUDES
prenom : MURIEL
prenom in Titre_conversation : False
nom : MILHAU
nom in Titre_conversation : False
tel : 99951616
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 3   temps = 0
Titre_conversationIFNTIFINDÉTUDES
prenom : MURIEL
prenom in Titre_conversation : False
nom : M

Titre_conversationIFNTIPROMOTION2021
prenom : SABIROU
prenom in Titre_conversation : False
nom : TEOURI
nom in Titre_conversation : False
tel : 90918141
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 3   temps = 0
Titre_conversationIFNTIPROMOTION2021
prenom : SABIROU
prenom in Titre_conversation : False
nom : TEOURI
nom in Titre_conversation : False
tel : 90918141
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
dans le while recherche de la conversation
click sur le bouton 0   temps = 1
Titre_conversationIFNTIPROMOTION2021
prenom : SABIROU
prenom in Titre_conversation : False
nom : TEOURI
nom in Titre_conversation : False
tel : 90918141
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
click sur le bouton 1   temps = 1
Titre_conversationIFNTIPROMOTION2021
prenom : SABIROU
prenom in Titre_conversation : False
nom : TEOURI
nom in Titre_conversation : Fal

0
1
2
3
4
dans le while recherche de la conversation
click sur le bouton 0   temps = 0
Titre_conversationSAHMICHAOU
prenom : SAHM
prenom in Titre_conversation : True
nom : ICHAOU
nom in Titre_conversation : True
tel : 90180359
str(tel) in Titre_conversation)False
('N' + num_contact) in Titre_conversation : False
Le titre correspond !
5
Titre conversation : Sahm Ichaou
 | contact trouvé 0 | 2 | 2.1 | contact sélectionné | aucun des mots-clés interdits n'a été trouvé -> envoi
Élément non trouvé pour le chemin : //footer//div[@contenteditable="true"][@dir="ltr"][@spellcheck="true"][@role="textbox"][@title="Taper un message"][@data-testid="conversation-compose-box-input"]
Terminé.
